In [1]:
# Install required packages
!pip install qiskit qiskit-aer matplotlib numpy scipy plotly pandas scikit-learn torch gymnasium stable-baselines3 pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 3.7 MB/s eta 0:00:00


In [2]:
import os
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set up the path for saving the model
path = "/content/drive/My Drive/Colab Notebooks/Quantum Realm/models/"
os.makedirs(path, exist_ok=True)


MessageError: Error: credential propagation was unsuccessful

In [ ]:



import io
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_state_qsphere
import torch
import torch.nn as nn
import torch.optim as optim


def generate_quantum_state(num_qubits):
    qc = QuantumCircuit(num_qubits)
    for _ in range(np.random.randint(1, 5)):
        qubit = np.random.randint(0, num_qubits)
        gate = np.random.choice(['h', 'x', 'y', 'z'])
        if gate == 'h':
            qc.h(qubit)
        elif gate == 'x':
            qc.x(qubit)
        elif gate == 'y':
            qc.y(qubit)
        else:
            qc.z(qubit)

    backend = Aer.get_backend('statevector_simulator')
    transpiled_qc = transpile(qc, backend)
    job = backend.run(transpiled_qc)
    result = job.result()
    statevector = result.get_statevector()
    return statevector

def quantum_state_to_image(statevector):
    fig, ax = plt.subplots(figsize=(5, 5))
    plot_state_qsphere(statevector, ax=ax)

    # Save the plot to a buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)

    # Convert the buffer to an image
    buf.seek(0)
    image = Image.open(buf)
    image = image.convert('RGB')

    # Convert to numpy array
    image_array = np.array(image)
    return image_array

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, num_channels=3):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, num_channels * 64 * 64),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 3, 64, 64)
        return img

class Discriminator(nn.Module):
    def __init__(self, num_channels=3):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(num_channels * 64 * 64, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# Generate dataset
num_qubits = 3
num_samples = 1000
quantum_dataset = []

for _ in range(num_samples):
    statevector = generate_quantum_state(num_qubits)
    image = quantum_state_to_image(statevector)
    quantum_dataset.append(image)

quantum_dataset = np.array(quantum_dataset)
quantum_dataset = (quantum_dataset - 127.5) / 127.5  # Normalize to [-1, 1]
quantum_dataset = quantum_dataset.transpose(0, 3, 1, 2)  # Change to (N, C, H, W) format

# Convert to PyTorch tensors
tensor_x = torch.Tensor(quantum_dataset)
dataloader = torch.utils.data.DataLoader(tensor_x, batch_size=64, shuffle=True)

# Initialize models and optimizers
latent_dim = 100
generator = Generator(latent_dim)
discriminator = Discriminator()

optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

criterion = nn.BCELoss()

# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    for i, imgs in enumerate(dataloader):
        batch_size = imgs.size(0)

        # Train Discriminator
        real_labels = torch.ones(batch_size, 1)
        fake_labels = torch.zeros(batch_size, 1)

        optimizer_D.zero_grad()
        outputs = discriminator(imgs)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        z = torch.randn(batch_size, latent_dim)
        fake_imgs = generator(z)
        outputs = discriminator(fake_imgs.detach())
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()

        d_loss = d_loss_real + d_loss_fake
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        z = torch.randn(batch_size, latent_dim)
        fake_imgs = generator(z)
        outputs = discriminator(fake_imgs)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_G.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}')

    # Save generated images
    if (epoch+1) % 10 == 0:
        z = torch.randn(16, latent_dim)
        generated_imgs = generator(z).detach().cpu().numpy()
        generated_imgs = (generated_imgs + 1) / 2.0  # Rescale to [0, 1]
        fig, axs = plt.subplots(4, 4, figsize=(10, 10))
        for i in range(4):
            for j in range(4):
                axs[i, j].imshow(generated_imgs[i*4+j].transpose(1, 2, 0))
                axs[i, j].axis('off')
        plt.savefig(os.path.join(path, f'generated_images_epoch_{epoch+1}.png'))
        plt.close()

# Save the trained models
torch.save(generator.state_dict(), os.path.join(path, 'generator.pth'))
torch.save(discriminator.state_dict(), os.path.join(path, 'discriminator.pth'))

In [ ]:

def quantum_state_to_latent(statevector, latent_dim=100):
    # This is a simplified mapping. In a more advanced system, you'd use a trained encoder.
    normalized_state = np.abs(statevector) / np.linalg.norm(np.abs(statevector))
    latent = np.random.randn(latent_dim)
    latent[:len(normalized_state)] = normalized_state
    return latent

# Load the trained generator
generator = Generator(latent_dim)
generator.load_state_dict(torch.load('generator.pth'))
generator.eval()

# Generate a quantum state
num_qubits = 3
quantum_state = generate_quantum_state(num_qubits)

# Map quantum state to latent space
latent_vector = quantum_state_to_latent(quantum_state, latent_dim)

# Generate image from latent vector
with torch.no_grad():
    latent_tensor = torch.FloatTensor(latent_vector).unsqueeze(0)
    generated_img = generator(latent_tensor).squeeze().permute(1, 2, 0).numpy()
    generated_img = (generated_img + 1) / 2.0  # Rescale to [0, 1]

# Visualize results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

plot_state_qsphere(quantum_state, ax=ax1)
ax1.set_title('Quantum State')

ax2.imshow(generated_img)
ax2.set_title('Generated Image')
ax2.axis('off')

plt.tight_layout()
plt.show()